In [20]:
!pip install -q beautifulsoup4 webdriver-manager

In [2]:
GAMEBOY_LP_URL = 'https://longplays.org/infusions/longplays/longplays.php?cat_id=30'

In [ ]:
!playwright install-deps

In [84]:
from playwright.async_api import async_playwright
import asyncio
from bs4 import BeautifulSoup
from urllib.parse import urljoin

async def scrape_site(url, selector='table'):
    async with async_playwright() as p:
        browser = await p.firefox.launch()
        page = await browser.new_page()
        await page.goto(url)
        # Wait for content to load
        await page.wait_for_selector(selector)
        content = await page.inner_html(selector)
        await browser.close()
        return content

In [61]:
content = await scrape_site(GAMEBOY_LP_URL, selector='tbody')

In [67]:
soup = BeautifulSoup(content, 'html.parser')

In [106]:
links = soup.find_all('a')
longplay_urls = {link.text: urljoin(GAMEBOY_LP_URL, link.get('href')) for link in links if 'longplay_id=' in link.get('href')}
urls = []
for link in links:
    if 'longplay_id=' in link.get('href'):
        urls.append((link.text, urljoin(GAMEBOY_LP_URL, link.get('href'))))

In [114]:
text_count = {}
for link in links:
    if 'longplay_id=' in link.get('href'):
        text = link.text
        text_count[text] = text_count.get(text, 0) + 1

# Print texts that appear more than once
duplicates = {text: count for text, count in text_count.items() if count > 1}

print("Duplicate link texts:", duplicates)

# To see the full entries with duplicate text:
for link in links:
    if 'longplay_id=' in link.get('href'):
        text = link.text
        if text in duplicates:
            print(f"Text: {text}")
            print(f"URL: {urljoin(GAMEBOY_LP_URL, link.get('href'))}")
            print("---")

Duplicate link texts: {'Alien vs Predator: The Last of His Clan': 2}
Text: Alien vs Predator: The Last of His Clan
URL: https://longplays.org/infusions/longplays/longplays.php?cat_id=30&longplay_id=13935
---
Text: Alien vs Predator: The Last of His Clan
URL: https://longplays.org/infusions/longplays/longplays.php?cat_id=30&longplay_id=13934
---


In [101]:
for url in urls:
    if url[0] == None:
        print(url)